In [2]:
#Environment set-up and libraries

#Base libraries
import numpy as np
import random
import torch
import torch.nn as nn

#Plotting libraries
%matplotlib inline
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go

#Utilities libraries
from glob import glob 
import os

import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
def load_file(file_name):
    print(file_name)

    if file_name.endswith(".las") or file_name.endswith(".laz"):
        print("[INFO] .las (.laz) file loading")
        try:
            # import lidar .las data and assign to variable
            pcd = laspy.read(file_name)
            # examine the available features for the lidar file we have read
            # list(las.point_format.dimension_names)
            #
            # set(list(las.classification))

            # Creating, Filtering, and Writing Point Cloud Data
            # To create 3D point cloud data, we can stack together with the X, Y, and Z dimensions, using Numpy like this.
            point_data = np.stack([pcd.X, pcd.Y, pcd.Z], axis=0).transpose((1, 0))
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_data)
            # points = point_data
            if pcd is not None:
                print("[Info] Successfully read", file_name)

                # Point cloud
                return pcd

        except Exception:
            print(".las, .laz file load failed")

    elif file_name.endswith(".e57"):
        print("[INFO] .e57 file loading")
        try:
            e57_file = pye57.E57(file_name)

            # other attributes can be read using:
            data = e57_file.read_scan(0)

            # 'data' is a dictionary with the point types as keys
            # assert isinstance(data["cartesianX"], np.ndarray)
            # assert isinstance(data["cartesianY"], np.ndarray)
            # assert isinstance(data["cartesianZ"], np.ndarray)

            point_xyz = np.stack([data["cartesianX"], data["cartesianY"], data["cartesianZ"]]).transpose((1, 0))
            # points_rgb = [data["colorRed"], data["colorGreen"], data["colorBlue"]]
            # points_intensity = data["intensity"]

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_xyz)
            # points = o3d.utility.Vector3dVector(point_xyz)
            # points = point_xyz
            # pcd.colors = o3d.utility.Vector3dVector(points_rgb)
            # pcd.colors[:, 0] = points_intensity
            print("[Info] Successfully read", file_name)
            return pcd

        except Exception:
            print(".e57 file load failed")

    elif file_name.endswith(".bin"):
        print("[INFO] .bin file loading")
        try:
            size_float = 4
            list_pcd = []
            with open(file_name, "rb") as f:
                byte = f.read(size_float * 4)
                while byte:
                    x, y, z, intensity = struct.unpack("ffff", byte)
                    list_pcd.append([x, y, z])
                    byte = f.read(size_float * 4)
            np_pcd = np.asarray(list_pcd)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(np_pcd)
            print("[Info] Successfully read", file_name)
            return pcd

        except Exception:
            print(".bin file load failed")

    elif file_name.endswith(".ply"):
        pcd = o3d.io.read_point_cloud(file_name)
        points_xyz = np.asarray(pcd.points)
        #pcd = o3d.geometry.PointCloud() # No need to do that already a PointCloud
        pcd.points = o3d.utility.Vector3dVector(points_xyz)
        # points = points_xyz
        if pcd is not None:
            print("[Info] Successfully read", file_name)
            # Point cloud
            return pcd

    elif file_name.endswith(".pts"):
        try:
            with open(file_name, "r") as f:
                # Log every 1000000 lines.
                LOG_EVERY_N = 1000000
                points_np = []
                for line in f:
                    if len(line.split()) == 4:
                        x, y, z, i = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z), float(i)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    elif len(line.split()) == 3:
                        x, y, z = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    elif len(line.split()) == 5:
                        x, y, z, i, zeroes_v = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z), float(i)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    elif len(line.split()) == 7:
                        x, y, z, r, g, b, i = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z),
                                          float(r), float(g), float(b),
                                          float(i)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    else:
                        print("[Info] The file has unregistered format")
                        return
            print('loop end')
            points_arr = np.array(points_np).transpose()
            print(len(points_arr))
            point_xyz = points_arr[:3].transpose()
            print("xyz points shape", point_xyz.shape)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_xyz)
            if len(points_arr) == 4:
                points_intensity = (points_arr[3])/255.0
                print("intensity points len", points_intensity.shape)
                points_intensity_rgb = np.vstack((points_intensity,
                                                  points_intensity,
                                                  points_intensity)).T
                print("intensity_rgb points shape", points_intensity_rgb.shape)
                pcd.colors = o3d.utility.Vector3dVector(points_intensity_rgb)
            elif len(points_arr) == 7:
                points_red = (points_arr[4]) / 255.0
                points_green = (points_arr[5]) / 255.0
                points_blue = (points_arr[6]) / 255.0
                points_rgb = np.vstack((points_red,
                                        points_green,
                                        points_blue)).T

                # points_intensity = ((points_arr[3]) / 255.0).T
                # print("intensity points len", points_intensity.shape)
                print("rgb points shape", points_rgb.shape)
                pcd.colors = o3d.utility.Vector3dVector(points_rgb)
                #pcd.intensities = o3d.utility.Vector3dVector(points_intensity)
            if pcd is not None:
                print("[Info] Successfully read", file_name)
                # Point cloud
                return pcd

        except Exception:
            print("[Info] Reading .pts file failed", file_name)

    # elif file_name.endswith(".kml"):
    #     try:
    #         with open(file_name, "r") as f:
    #             # Log every 1000000 lines.
    #             LOG_EVERY_N = 1000000
    #             points_np = []
    #             for line in f:
    #                 print(line)
    #                 if len(line.split(",")) == 3 and (line[0].isdigit() or line.startswith("-")):
    #                     y, x, z = [num for num in line.split(",")]
    #                     points_np.append([float(x), float(y), float(z)])
    #                     if (len(points_np) % LOG_EVERY_N) == 0:
    #                         print('point', len(points_np))
    #                 else:
    #                     print("[Info] The file has unregistered format")
    #         print('loop end')
    #         points_arr = np.array(points_np).transpose()
    #         print(len(points_arr))
    #         point_xyz = points_arr[:3].transpose()
    #         # points_intensity = points_arr[3]
    #         pcd = o3d.geometry.PointCloud()
    #         pcd.points = o3d.utility.Vector3dVector(point_xyz)
    #         if pcd is not None:
    #             print("[Info] Successfully read", file_name)
    #             # Point cloud
    #             return pcd
    #
    #     except Exception:
    #         print("[Info] Reading .kml file failed", file_name)

    else:
        pcd = None
        geometry_type = o3d.io.read_file_geometry_type(file_name)
        print(geometry_type)

        mesh = None
        if geometry_type & o3d.io.CONTAINS_TRIANGLES:
            mesh = o3d.io.read_triangle_model(file_name)
        if mesh is None:
            print("[Info]", file_name, "appears to be a point cloud")
            cloud = None
            try:
                cloud = o3d.io.read_point_cloud(file_name)
                # print(type(cloud))
            except Exception:
                print("[Info] Unknown filename", file_name)
            if cloud is not None:
                print("[Info] Successfully read", file_name)

                if not cloud.has_normals():
                    cloud.estimate_normals()
                cloud.normalize_normals()
                pcd = cloud
                #points = cloud.points
                pcd.points = o3d.utility.Vector3dVector(cloud.points)
            else:
                print("[WARNING] Failed to read points", file_name)

        if pcd is not None or mesh is not None:
            try:
                if mesh is not None:
                    # Triangle model
                    _scene.scene.add_model("__model__", mesh)
                else:
                    # Point cloud
                    return pcd

            except Exception as e:
                print(e)


In [4]:
filepath_mob1 = "/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE ADVANCED 3D - Selected MMS Data/"
filename1 = "SABRE MMS_S3 - 0002.pts"
filepath1 = filepath_mob1 + filename1
pc1 = load_file(filepath1)

filepath_static2 = "/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE - Selected Static Scan Data/"
filename2 = "SABRE Static Scan_T17_003.pts"
filepath2 = filepath_static2 + filename2
pc2 = load_file(filepath2)

/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE ADVANCED 3D - Selected MMS Data/SABRE MMS_S3 - 0002.pts
point 1000000
point 2000000
point 3000000
point 4000000
point 5000000
point 6000000
point 7000000
point 8000000
point 9000000
point 10000000
point 11000000
loop end
4
xyz points shape (11008825, 3)
intensity points len (11008825,)
intensity_rgb points shape (11008825, 3)
[Info] Successfully read /home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE ADVANCED 3D - Selected MMS Data/SABRE MMS_S3 - 0002.pts
/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE - Selected Static Scan Data/SABRE Static Scan_T17_003.pts
point 1000000
point 2000000
point 3000000
point 4000000
point 5000000
point 6000000
point 7000000
point 8000000
point 9000000
point 10000000
point 11000000
point 12000000
point 13000000
point 14000000
point 15000000
point 16000000
point 17

In [5]:
def create_voxel_boxes(point_cloud, num_voxels):
    """
    Create voxel boxes for a given point cloud and number of voxels along each axis.

    Parameters:
    - point_cloud: Nx3 numpy array representing the point cloud.
    - num_voxels: List or tuple with the number of voxels along each axis (x, y, z).

    Returns:
    - voxel_boxes: List of Open3D OrientedBoundingBox objects.
    """
    
    # Compute the bounding box of the point cloud
    min_bound = np.min(point_cloud, axis=0)
    max_bound = np.max(point_cloud, axis=0)

    # Determine the voxel size in each dimension
    voxel_size = (max_bound - min_bound) / num_voxels
    print(f"Voxel size: {voxel_size}/n")

    voxel_boxes = []

    for i in range(num_voxels[0]):
        for j in range(num_voxels[1]):
            for k in range(num_voxels[2]):
                # Compute the center of the voxel
                center = min_bound + voxel_size * (np.array([i, j, k]) + 0.5)
                # Create an oriented bounding box for the voxel
                obb = o3d.geometry.OrientedBoundingBox(center=center, R=np.eye(3), extent=voxel_size)
                voxel_boxes.append(obb)
                
    voxel_dict = {}

    for point in point_cloud:
        # Determine the voxel index for the point
        voxel_index = tuple(((point - min_bound) // voxel_size).astype(int))
        
        if voxel_index not in voxel_dict:
            voxel_dict[voxel_index] = []
        
        voxel_dict[voxel_index].append(point)

    return voxel_boxes, voxel_dict

In [6]:
def visualize_point_cloud_with_voxels(point_cloud, voxel_boxes):
    """
    Visualize a point cloud alongside voxel boxes.

    Parameters:
    - point_cloud: Nx3 numpy array representing the point cloud.
    - voxel_boxes: List of Open3D OrientedBoundingBox objects.
    """
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    print("Number of voxels:", len(voxel_boxes))

    o3d.visualization.draw_geometries([pcd, *voxel_boxes])
    
#     #Visualize
#     o3d.visualization.draw_plotly([pcd, *voxel_boxes],point_sample_factor=0.5, width=600, height=400)
#     visualizer = JVisualizer()
#     visualizer.add_geometry(pcd)
#     visualizer.add_geometry(*voxel_boxes)
#     visualizer.show()
    
    #draw(pcd)


In [7]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                             max_nn=30))
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0,
                                             max_nn=100))
    return pcd_down, pcd_fpfh

In [11]:
# Downsampling
voxel_size = 0.3

pc1_down, pc1_fpfh = preprocess_point_cloud(pc1, voxel_size)
pc1_points = np.asarray(pc1_down.points)
print(pc1_points.shape)

pc2_down, pc2_fpfh = preprocess_point_cloud(pc2, voxel_size)
pc2_points = np.asarray(pc2_down.points)
print(pc2_points.shape)


(56746, 3)
(123234, 3)


In [13]:
print(pc1_fpfh)
print(pc2_fpfh)

Feature class with dimension = 33 and num = 56746
Access its data via data member.
Feature class with dimension = 33 and num = 123234
Access its data via data member.


In [14]:

num_voxels = [150, 300, 1]
voxel_boxes, voxel_dict = create_voxel_boxes(pc1_points, num_voxels)

# Count the number of points in each voxel
for voxel_index, points in voxel_dict.items():
    print(f"Voxel {voxel_index} contains {len(points)} points.")
    
# visualize_point_cloud_with_voxels(pc1_points, voxel_boxes)

Voxel size: [ 1.02727621  0.72453667 36.09      ]/n
Voxel (10, 186, 0) contains 20 points.
Voxel (20, 116, 0) contains 2 points.
Voxel (10, 185, 0) contains 4 points.
Voxel (21, 113, 0) contains 11 points.
Voxel (21, 103, 0) contains 29 points.
Voxel (19, 125, 0) contains 2 points.
Voxel (20, 102, 0) contains 12 points.
Voxel (21, 108, 0) contains 7 points.
Voxel (19, 127, 0) contains 4 points.
Voxel (8, 191, 0) contains 9 points.
Voxel (10, 191, 0) contains 6 points.
Voxel (20, 101, 0) contains 2 points.
Voxel (18, 131, 0) contains 4 points.
Voxel (10, 192, 0) contains 6 points.
Voxel (6, 186, 0) contains 3 points.
Voxel (19, 128, 0) contains 4 points.
Voxel (19, 130, 0) contains 36 points.
Voxel (20, 117, 0) contains 3 points.
Voxel (21, 110, 0) contains 9 points.
Voxel (21, 114, 0) contains 8 points.
Voxel (8, 186, 0) contains 17 points.
Voxel (21, 104, 0) contains 20 points.
Voxel (21, 105, 0) contains 10 points.
Voxel (9, 185, 0) contains 15 points.
Voxel (7, 186, 0) contains 11 p

In [15]:
# Calculating voxel size for each voxel (in number of points) 
# and finding voxel with the max number of points
max_vox_size = 0 
key_max_vox = ""

for k in voxel_dict.keys(): 
    print(len(voxel_dict[k])) 
    if max_vox_size < len(voxel_dict[k]): 
        max_vox_size = len(voxel_dict[k]) 
        key_max_vox = k 
print('\n') 
print("Max points in voxel:", max_vox_size) 
print("Vox with Max points index:", key_max_vox)
print("Number of voxels:", len(voxel_dict.keys()))
print("Avg number of points in voxel:", len(pc1_points)//len(voxel_dict.keys()))

20
2
4
11
29
2
12
7
4
9
6
2
4
6
3
4
36
3
9
8
17
20
10
15
11
3
1
4
6
12
7
4
2
20
4
9
2
16
2
3
5
8
11
6
15
12
12
12
4
4
6
8
11
8
6
12
5
17
41
7
7
8
6
6
15
1
8
4
6
6
12
20
5
14
11
8
4
9
8
6
37
18
6
12
9
1
6
8
1
11
15
6
4
6
8
2
9
12
18
9
29
9
6
15
14
16
14
12
9
1
29
6
9
8
9
26
39
2
6
9
7
11
6
1
4
8
6
11
12
8
8
4
12
12
4
4
9
8
18
8
9
12
6
22
3
2
12
10
6
27
6
17
17
15
10
22
9
7
8
6
12
8
6
7
6
13
21
9
8
3
3
9
12
12
6
9
9
37
6
11
4
17
32
9
14
14
12
18
8
2
3
8
31
6
2
2
12
6
8
10
6
6
1
3
12
6
12
14
33
8
8
19
9
9
9
8
12
13
7
6
8
5
11
11
2
36
6
9
9
6
8
8
12
6
8
12
11
18
8
1
6
9
17
17
6
6
19
9
30
7
9
6
7
17
31
15
7
11
15
9
9
6
6
1
18
15
6
34
6
6
14
7
12
6
9
16
9
6
7
7
9
6
9
10
11
11
6
6
40
9
8
6
12
27
8
8
10
17
16
9
8
6
9
18
12
9
10
19
8
6
12
14
4
2
4
15
8
7
1
12
9
7
8
9
8
18
15
3
6
11
25
15
1
20
10
6
9
6
12
6
14
24
12
3
19
6
9
14
5
3
13
8
12
9
12
5
1
8
4
4
1
6
9
6
14
1
41
16
12
9
7
4
8
15
40
1
13
8
9
13
17
12
13
6
12
9
2
11
2
6
3
8
11
9
16
8
10
5
8
2
17
11
12
9
8
10
14
12
11
10
8
13
14
5
14
12
38


In [16]:

num_voxels = [500, 900, 1]
voxel_boxes2, voxel_dict2 = create_voxel_boxes(pc2_points, num_voxels)

# Count the number of points in each voxel
for voxel_index, points in voxel_dict2.items():    
    print(f"Voxel {voxel_index} contains {len(points)} points.")
    
#visualize_point_cloud_with_voxels(pc2_points, voxel_boxes2)

Voxel size: [ 1.036268    0.35370667 28.81333333]/n
Voxel (130, 414, 0) contains 1 points.
Voxel (161, 412, 0) contains 4 points.
Voxel (337, 403, 0) contains 2 points.
Voxel (341, 403, 0) contains 3 points.
Voxel (162, 412, 0) contains 14 points.
Voxel (339, 404, 0) contains 5 points.
Voxel (353, 403, 0) contains 4 points.
Voxel (164, 412, 0) contains 1 points.
Voxel (142, 413, 0) contains 1 points.
Voxel (211, 409, 0) contains 4 points.
Voxel (219, 409, 0) contains 3 points.
Voxel (302, 405, 0) contains 4 points.
Voxel (130, 413, 0) contains 3 points.
Voxel (131, 413, 0) contains 5 points.
Voxel (132, 413, 0) contains 3 points.
Voxel (217, 409, 0) contains 4 points.
Voxel (335, 404, 0) contains 6 points.
Voxel (354, 403, 0) contains 2 points.
Voxel (358, 403, 0) contains 1 points.
Voxel (351, 403, 0) contains 3 points.
Voxel (213, 409, 0) contains 5 points.
Voxel (350, 403, 0) contains 3 points.
Voxel (161, 411, 0) contains 2 points.
Voxel (308, 405, 0) contains 5 points.
Voxel (462,

In [17]:

max_val = 0 
teacher = ""

for k in voxel_dict2.keys(): 
    print(len(voxel_dict2[k])) 
    if max_val < len(voxel_dict2[k]): 
        max_val = len(voxel_dict2[k]) 
        teacher = k 
print('\n') 
print(max_val) 
print(teacher)
print(len(voxel_dict2.keys()))
print(len(pc2_points)//len(voxel_dict2.keys()))

1
4
2
3
14
5
4
1
1
4
3
4
3
5
3
4
6
2
1
3
5
3
2
5
8
8
15
2
1
1
2
1
3
5
3
3
4
6
1
1
2
4
3
6
5
2
7
7
3
8
1
3
3
7
1
4
8
5
3
4
5
6
5
2
9
5
4
4
7
1
2
4
4
4
3
5
2
8
4
2
13
2
1
2
8
3
5
36
3
3
2
1
5
5
5
6
4
2
6
6
31
1
8
2
9
3
5
5
7
6
4
3
3
10
4
5
3
7
2
5
7
4
5
5
7
5
3
3
4
1
2
6
3
5
2
3
24
3
2
2
3
7
5
4
8
4
4
1
12
2
2
7
7
5
17
2
4
3
2
3
7
1
7
6
4
6
5
2
2
9
3
4
3
7
1
5
11
5
16
22
1
2
2
3
7
7
4
7
5
4
4
14
4
5
2
4
28
9
2
3
2
2
3
2
4
8
7
4
9
4
1
4
5
3
3
4
3
17
4
1
2
2
5
5
7
7
4
5
5
4
13
3
1
11
2
2
1
3
6
6
2
5
26
6
3
3
2
3
5
3
7
2
2
3
5
4
6
2
2
2
3
7
6
3
3
2
3
1
2
1
3
3
8
1
3
3
7
2
2
2
2
5
1
4
6
3
7
7
11
2
3
4
6
19
5
3
2
2
1
4
4
1
2
3
3
5
8
3
11
1
2
1
4
5
3
6
11
5
9
3
4
3
1
2
2
3
5
1
5
5
29
5
3
3
2
1
2
3
8
4
2
4
5
2
2
4
2
4
12
7
3
3
6
4
1
5
36
7
4
1
4
3
7
5
5
1
3
2
4
3
3
4
4
3
2
4
1
3
1
3
3
7
5
3
9
1
3
2
2
2
2
4
5
5
4
3
4
9
3
4
2
1
37
6
8
4
2
1
3
3
3
10
2
4
4
3
2
2
4
4
4
2
1
3
1
1
2
4
1
5
2
3
4
1
4
4
5
3
4
4
1
4
3
4
3
14
2
3
4
3
4
3
4
3
3
2
6
2
2
3
12
4
2
5
1
3
4
4
2
8
5
3
1
1
6
7
4
2
2
4
3
1
7
2
4
3

In [18]:
def compute_chamfer_distance(pcd1, pcd2):
    """
    Compute the Chamfer distance between two point clouds.

    Parameters:
    - pcd1, pcd2: Open3D point cloud objects.

    Returns:
    - chamfer_distance: The Chamfer distance between the two point clouds.
    """
    
    # Compute distance from pcd1 to pcd2
    distances_1_to_2 = pcd1.compute_point_cloud_distance(pcd2)
    avg_distance_1_to_2 = np.mean([np.min(dist) for dist in distances_1_to_2])

    # Compute distance from pcd2 to pcd1
    distances_2_to_1 = pcd2.compute_point_cloud_distance(pcd1)
    avg_distance_2_to_1 = np.mean([np.min(dist) for dist in distances_2_to_1])

    # Compute the Chamfer distance
    chamfer_distance = (avg_distance_1_to_2 + avg_distance_2_to_1) / 2

    return chamfer_distance

In [19]:
points1_mean = pc1_points.mean(axis=0)
points2_mean = pc2_points.mean(axis=0)
pc1_points = (pc1_points - points1_mean)
pc2_points = (pc2_points - points2_mean)
print(pc1_points, pc2_points)

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pc1_points)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pc2_points)

chamfer_dist = compute_chamfer_distance(pcd1, pcd2)
print(f"Chamfer Distance: {chamfer_dist}")

[[-79.4228399   37.30858662   6.72593767]
 [-79.70144899  37.47339571   7.18775586]
 [-69.1670497  -13.17859181  -0.53335644]
 ...
 [-28.3096599  -13.77843338  -0.47534233]
 [-28.21018276 -14.59315624  -0.58477661]
 [-28.19878593 -14.78524354  -0.55565756]] [[-152.3667831    43.66076066    6.2836434 ]
 [-120.4017831    43.10576066   -3.9893566 ]
 [  62.0582169    39.99026066   -7.7433566 ]
 ...
 [ -23.07166947   35.70601066   -6.63479979]
 [  14.72273078   17.97155232   -4.99692605]
 [ -22.77844977   16.15742732   -6.57282327]]
Chamfer Distance: 17.04458131853736


In [20]:
# # Compute chamfer distance for voxels from both point clouds
# for voxel_index, points in voxel_dict.items():
#     if len(points)>100000:
#         for voxel_index2, points2 in voxel_dict2.items():
#             if len(points2)>100000:
        
#     print(f"Voxel {voxel_index} contains {len(points)} points.")

In [21]:
# Define the Chamfer distance loss function
def chamfer_distance(p1, p2):
    p1 = p1.unsqueeze(1)  # Shape: (B, 1, N, 3)
    p2 = p2.unsqueeze(0)  # Shape: (1, B, M, 3)
    
    p1_repeat = p1.repeat(1, 1, 1, p2.shape[3])  # Shape: (B, B, N, 3)
    p2_repeat = p2.repeat(1, 1, p1.shape[2], 1)  # Shape: (B, B, N, 3)
    
    distances = torch.norm(p1_repeat - p2_repeat, dim=3, p=2)  # Shape: (B, B, N)
    
    min_distances_p1, _ = torch.min(distances, dim=2)  # Shape: (B, B)
    min_distances_p2, _ = torch.min(distances, dim=1)  # Shape: (B, B)
    
    chamfer_dist = torch.mean(min_distances_p1, dim=1) + torch.mean(min_distances_p2, dim=1)  # Shape: (B,)
    
    return chamfer_dist

In [22]:
# Convert NumPy arrays to PyTorch tensors
source_tensor = torch.FloatTensor(pc1_points)
target_tensor = torch.FloatTensor(pc2_points)

In [23]:
# Define voxelization parameters
voxel_size = 0.3

In [24]:
# Voxel grid dimensions (customize as needed)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd1 + pcd2, voxel_size)
min_bound, max_bound = voxel_grid.get_min_bound(), voxel_grid.get_max_bound()
voxel_grid_size = ((max_bound - min_bound) / voxel_size).astype(int)

In [25]:
# Function to map points to voxel indices
def points_to_voxel_indices(points, voxel_size, min_bound):
    indices = ((points - min_bound) / voxel_size).astype(int)
    return indices

In [26]:
# Map points to voxel indices
source_indices = points_to_voxel_indices(pc1_points, voxel_size, min_bound)
target_indices = points_to_voxel_indices(pc2_points, voxel_size, min_bound)

In [27]:
# Create tensors to hold voxelized point clouds
source_voxels = torch.zeros(voxel_grid_size[0], voxel_grid_size[1], voxel_grid_size[2], dtype=torch.float32)
target_voxels = torch.zeros(voxel_grid_size[0], voxel_grid_size[1], voxel_grid_size[2], dtype=torch.float32)

In [28]:
# Assign points to voxels
source_voxels[source_indices[:, 0], source_indices[:, 1], source_indices[:, 2]] = 1.0  # Set occupancy to 1
target_voxels[target_indices[:, 0], target_indices[:, 1], target_indices[:, 2]] = 1.0  # Set occupancy to 1


In [29]:
# Define batch size
batch_size = 8

In [30]:
# Split voxelized point clouds into batches
source_batches = torch.split(source_voxels.unsqueeze(0), batch_size)
target_batches = torch.split(target_voxels.unsqueeze(0), batch_size)


In [31]:
# Concat source and target batches into tensors
source_data = torch.cat(source_batches, dim=0)
target_data = torch.cat(target_batches, dim=0)

In [32]:
from torch.utils.data import DataLoader, TensorDataset

In [33]:
# Ensure the feature dimensions of the source and target data match d_model
d_model = source_data.size(-1)  # Use the feature dimension of the data as d_model


In [34]:
# Combine source and target batches into a dataset
dataset = TensorDataset(source_data, target_data)

In [35]:
# class TransformerEncoder(nn.Module):
#     def __init__(self, d_model, nhead, num_encoder_layers):
#         super(TransformerEncoder, self).__init__()
#         self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers)
#         self.fc = nn.Linear(3, d_model)

#     def forward(self, src):
#         src = self.fc(src)
#         return self.transformer.encoder(src)

# class TransformerDecoder(nn.Module):
#     def __init__(self, d_model, nhead, num_decoder_layers):
#         super(TransformerDecoder, self).__init__()
#         self.transformer = nn.Transformer(d_model, nhead, num_decoder_layers)
#         self.fc = nn.Linear(d_model, 3)

#     def forward(self, src, memory):
#         src = self.transformer.decoder(src, memory)
#         return self.fc(src)

# class PointCloudTransformer(nn.Module):
#     def __init__(self, d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6):
#         super(PointCloudTransformer, self).__init__()
#         self.encoder = TransformerEncoder(d_model, nhead, num_encoder_layers)
#         self.decoder = TransformerDecoder(d_model, nhead, num_decoder_layers)

#     def forward(self, src, tgt):
#         memory = self.encoder(src)
#         output = self.decoder(tgt, memory)
#         return output

# A simple Transformer-based registration model
class TransformerRegistration(nn.Module):
    def __init__(self, d_model):
        super(TransformerRegistration, self).__init__()

        # Define the Transformer layers here (customize as needed)
        self.transformer = nn.Transformer(d_model=d_model, nhead=d_model, num_encoder_layers=4, num_decoder_layers=4)

        # Add any additional layers for point cloud registration (e.g., FC layers)
        self.fc = nn.Linear(d_model, 3)  # 3 for 3D transformation vector

    def forward(self, source, target):
        # Flatten source and target for Transformer input
        source = source.view(source.size(0), -1, source.size(-1))
        target = target.view(target.size(0), -1, target.size(-1))
        
        # Perform point cloud registration using Transformer
        source_transformed = self.transformer(source, target)

        # Apply any additional layers for registration refinement
        transformation = self.fc(source_transformed)

        return transformation

In [36]:
# Initialize the transformer model and Define the optimizer
# model = PointCloudTransformer()
model = TransformerRegistration(d_model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [37]:
# Create a data loader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [ ]:
# output = model(source_tensor, target_tensor)


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for source_batch, target_batch in dataloader:
        # Forward pass through the model
        transformed_points = model(source_batch, target_batch)

        # Calculate Chamfer distance loss
        loss = chamfer_distance(transformed_points, target_data)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print loss for monitoring
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()}")
#     if (epoch + 1) % 100 == 0:
#         print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()}")


In [ ]:
# Align the source point cloud using the learned transformation
aligned_source_points = src_encoded.detach().numpy()

# Convert aligned source points back to Open3D format for visualization
aligned_source_pcd = o3d.geometry.PointCloud()
aligned_source_pcd.points = o3d.utility.Vector3dVector(aligned_source_points)

# Visualize the aligned point cloud
o3d.visualization.draw_geometries([aligned_source_pcd, target_pcd])